In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/Cls/data.nosync18/")
classpath = os.path.join(projectdir, "class3")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
#mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 
mu_integral_step = 1.00


# Generate output paths  
#ps18_resultsdir = os.path.join(projectdir, 'results', 'ps18')
ps18_resultsdir = os.path.join("/Volumes/SSD01/Cls/", 'results', 'ps18')


#ps18_convergencedir = os.path.join(ps18_resultsdir, 'convergence')
cf.makedirectory(ps18_resultsdir)
#cf.makedirectory(ps18_convergencedir) 

In [2]:

# Linda Fiducial Cosmology 
ps18_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (1.5072/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
#z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
z_table = np.array([0.65, 0.75])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])

omegacdm_set = np.array([ps18_fid['omega_cdm'] - ps18_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                                  
ps18_fiducialset = [[dict(ps18_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[tidx, midx], 'T_ncdm' : temps[tidx]/2.726}) 
               for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)] 
ps18_forecastset = [[cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fidrowvals)] for fidrowidx, fidrowvals in enumerate(ps18_fiducialset)]
for frowidx, frowval in enumerate(ps18_forecastset): 
    for fidx, fcst in enumerate(frowval): 
        fcst.gen_pm()
        fcst.gen_fisher(
            fisher_order=[
                'omega_b',                                    
                'omega_cdm',                                  
                'n_s',                                        
                'A_s',                                        
                'tau_reio',                                   
                'h',                                                                             
                'N_ncdm',                                 
                'sigma_fog',                                   
                'b0',                                         
                'alpha_k2'],
            mu_step=mu_integral_step, 
            skipgen=False)
        print("Relic Forecast ", fidx, " complete...")
        dill.dump_session(os.path.join(ps18_resultsdir, 'Apr3_'+str(frowidx)+'_'+str(fidx)+'_ps18.db'))

Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851289.2166930.306997
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851297.7360420.376698
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851306.6234830.573663
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851315.08179620.221008
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851324.156210.527887
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851333.9003160.344865
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851342.70416520.797409
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851351.5373090.966465
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851360.8990430.828850
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851369.8211460.060638
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851378.0066620.496651
Dataset already exists at: /Volumes/SSD01/Cls/data.no

/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/forecast.py:803: RuntimeWarning: invalid value encountered in true_divide
  dlogPdmu = (logP_mu_high - logP_mu_low) / (2. * self.dstep * mu)


Fisher element ( 0 ,  0 ), ( 0 ,  0 ) calculated...
Fisher element ( 0 ,  1 ), ( 1 ,  0 ) calculated...
Fisher element ( 0 ,  2 ), ( 2 ,  0 ) calculated...
Fisher element ( 0 ,  3 ), ( 3 ,  0 ) calculated...
Fisher element ( 0 ,  4 ), ( 4 ,  0 ) calculated...
Fisher element ( 0 ,  5 ), ( 5 ,  0 ) calculated...
Fisher element ( 0 ,  6 ), ( 6 ,  0 ) calculated...
Fisher element ( 0 ,  7 ), ( 7 ,  0 ) calculated...
Fisher element ( 0 ,  8 ), ( 8 ,  0 ) calculated...
Fisher element ( 0 ,  9 ), ( 9 ,  0 ) calculated...
Fisher element ( 1 ,  1 ), ( 1 ,  1 ) calculated...
Fisher element ( 1 ,  2 ), ( 2 ,  1 ) calculated...
Fisher element ( 1 ,  3 ), ( 3 ,  1 ) calculated...
Fisher element ( 1 ,  4 ), ( 4 ,  1 ) calculated...
Fisher element ( 1 ,  5 ), ( 5 ,  1 ) calculated...
Fisher element ( 1 ,  6 ), ( 6 ,  1 ) calculated...
Fisher element ( 1 ,  7 ), ( 7 ,  1 ) calculated...
Fisher element ( 1 ,  8 ), ( 8 ,  1 ) calculated...
Fisher element ( 1 ,  9 ), ( 9 ,  1 ) calculated...
Fisher eleme

Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851802.3309550.080702
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851810.5798820.114546
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851818.5865650.016259
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851826.54727980.451851
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851834.4295820.757857
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851842.37581010.297473
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851850.4592540.616057
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851855.09913180.184346
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851859.710350.181090
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851867.876530.792060
Dataset already exists at: /Volumes/SSD01/Cls/data.nosync18/1585851875.82004620.696258
Dataset already exists at: /Volumes/SSD01/Cls/data.n

Fisher element ( 2 ,  2 ), ( 2 ,  2 ) calculated...
Fisher element ( 2 ,  3 ), ( 3 ,  2 ) calculated...
Fisher element ( 2 ,  4 ), ( 4 ,  2 ) calculated...
Fisher element ( 2 ,  5 ), ( 5 ,  2 ) calculated...
Fisher element ( 2 ,  6 ), ( 6 ,  2 ) calculated...
Fisher element ( 2 ,  7 ), ( 7 ,  2 ) calculated...
Fisher element ( 2 ,  8 ), ( 8 ,  2 ) calculated...
Fisher element ( 2 ,  9 ), ( 9 ,  2 ) calculated...
Fisher element ( 3 ,  3 ), ( 3 ,  3 ) calculated...
Fisher element ( 3 ,  4 ), ( 4 ,  3 ) calculated...
Fisher element ( 3 ,  5 ), ( 5 ,  3 ) calculated...
Fisher element ( 3 ,  6 ), ( 6 ,  3 ) calculated...
Fisher element ( 3 ,  7 ), ( 7 ,  3 ) calculated...
Fisher element ( 3 ,  8 ), ( 8 ,  3 ) calculated...
Fisher element ( 3 ,  9 ), ( 9 ,  3 ) calculated...
Fisher element ( 4 ,  4 ), ( 4 ,  4 ) calculated...
Fisher element ( 4 ,  5 ), ( 5 ,  4 ) calculated...
Fisher element ( 4 ,  6 ), ( 6 ,  4 ) calculated...
Fisher element ( 4 ,  7 ), ( 7 ,  4 ) calculated...
Fisher eleme

...

Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585974988.17531010.382280
#A_s-->A_s = 2.2499568e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.10355821088252129
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 5.011872336272719
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585974995.6131870.775728
#A_s-->A_s = 2.2142432e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.10355821088252129
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 5.011872336272719
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975003.0393960.622924
#A_s-->A_s = 2.2142432e-09
#n_s-->n_s = 0.967


#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.022081919999999998
#omega_cdm-->omega_cdm = 0.0997868796465376
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 7.07945784384138
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975098.5247190.875375
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.1005851746837099
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 7.07945784384138
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975102.749370.645336
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.1005851746837099
#tau_reio-

Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975212.3375330.750894
#A_s-->A_s = 2.2142432e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.0997868796465376
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 7.07945784384138
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975219.18611720.928705
#A_s-->A_s = 2.2142432e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.0997868796465376
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 7.07945784384138
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975226.0717380.755720
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.97473599999

Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975326.372430.234756
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.09521541054396213
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 10.0
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975330.6482690.389446
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.0937040548210421
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 10.0
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975335.0052740.544755
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cd

Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975455.0344840.483363
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.9747359999999999
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.09445973268250211
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 10.0
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975462.3781750.978270
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.959264
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.09445973268250211
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1
#T_ncdm-->T_ncdm = 0.39618488628026416
#m_ncdm-->m_ncdm = 10.0
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/Cls/data.nosync18/1585975469.59167220.037716
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.959264
#omega_b-->omega

In [3]:
import dill
import os
ps18_resultsdir = os.path.join("/Volumes/SSD01/Cls/", 'results', 'ps18')
dill.load_session(os.path.join(ps18_resultsdir, 'Apr3_3_20_ps18.db'))

In [1]:
for frowidx, frowval in enumerate(ps18_forecastset): 
    for fidx, fcst in enumerate(frowval): 
        print(type(fcst.fisher))

NameError: name 'ps18_forecastset' is not defined